In [1]:
import pandas as pd
import numpy as np
from glob import glob
# from tqdm import tqdm
from tqdm import notebook
import mysql.connector
# from sqlalchemy import create_engine

import re
import json

In [2]:
# DB설정
with open('./path.json') as file:
    json_obj = json.load(file)
    
con = mysql.connector.connect(host = json_obj['DB']['Host'],
                              port = json_obj['DB']['Port'],
                              database = 'capstone',
                              user = json_obj['DB']['User'],
                              password = str(json_obj['DB']['Password'])
)

In [3]:
cursor = con.cursor(dictionary=True)

In [4]:
cursor.execute('show Databases;')
result = cursor.fetchall()

In [5]:
pd.DataFrame(result)

,Database
0,capstone
1,information_schema
2,mysql
3,performance_schema
4,sys


In [6]:
apt = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
officetel = pd.read_csv("./data/실거래_가공/오피스텔_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
yeon = pd.read_csv('./data/실거래_가공/연립다세대_실거래_10년_가공.csv',index_col = 0, encoding='cp949')

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_68947/1795912215.py:1: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  apt = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_68947/1795912215.py:2: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  officetel = pd.read_csv("./data/실거래_가공/오피스텔_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_68947/1795912215.py:3: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  yeon = pd.read_csv('./data/실거래_가공/연립다세대_실거래_10년_가공.csv',index_col = 0, encoding='cp949')


In [7]:
print(len(apt))
print(len(officetel))
print(len(yeon))

print(set(apt.columns).difference(set(yeon.columns))) # 단지명
print(set(yeon.columns).difference(set(apt.columns))) # 건물명

2024191
447133
1073939
{'단지명'}
{'건물명'}


In [8]:
import re
import json

re1 = re.compile("^(수원|고양|용인|창원|성남|청주|전주|천안|안산|안양|포항)")
re2 = re.compile("([읍면동]|[0-9]가|시장북로|세종로|남성로)$")


In [9]:
def parse(address, how='jibun'):
    '''
    주소를 받고 how 값에 따라 지번 또는 도로명 주소로 값을 return하는 메소드

    address : 주소
    how : 지번으로 parsing 또는 도로명으로 parsing ('jibun', 'road')
    '''
    parts = address.split()
    sido = ""
    sigungu = ""
    ilbangu = ""
    eupmyeondong = ""
    ri_roadname = ""
    for idx, part in enumerate(parts):
        if idx == 0:
            sido = part
            continue
        
        if part.endswith("시") or part.endswith("군"):
            sigungu = part
            continue
        
        if part.endswith("구"):
            if idx == 1:
                if len(part) >= 4 and re.search(re1, part):
                    sigungu = part[0:2] + "시"
                    ilbangu = part[2:]
                else:
                    sigungu = part
            else:
                ilbangu = part
            continue
        
        if re.search(re2,
                        part) and eupmyeondong == '' and how == 'jibun':
            eupmyeondong = part
            continue
            
        if part.endswith("리") and how == 'jibun':
            ri_roadname = part
            continue
        
        if (part.endswith('로') or part.endswith('길')
            ) and how == 'road' and ri_roadname == '':
            ri_roadname = part
            continue
        
        if re.search(r"([가-힣]{2,})(\([^0-9가-힣a-zA-Z]{1,}\))", part):
            ri_roadname = part
            continue

    return sido, sigungu, ilbangu, eupmyeondong, ri_roadname


In [10]:
apt[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(apt['시군구'].iloc[i]) for i in notebook.tqdm(range(len(apt)))]
officetel[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(officetel['시군구'].iloc[i]) for i in notebook.tqdm(range(len(officetel)))]
yeon[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(yeon['시군구'].iloc[i]) for i in notebook.tqdm(range(len(yeon)))]

  0%|          | 0/2024191 [00:00<?, ?it/s]

  0%|          | 0/447133 [00:00<?, ?it/s]

  0%|          | 0/1073939 [00:00<?, ?it/s]

In [11]:
apt.columns
t = apt.sample(50)

('토지고유번호',
 'sido',
 'sigungu',
 'ilbangu',
 'eupmyeondong',
 'ri_roadname',
 '본번',
 '부번',
 '산',
 '단지명',
 '전월세구분',
 '전용면적(㎡)',
 '계약년월',
 '계약일',
 '보증금(만원)',
 '월세(만원)',
 '층',
 '건축년도',
 '도로명',
 '계약기간')

In [27]:
apt['계약기간'].sample(5)

1071460                -
1529505                -
750788                 -
1473370    202111~202311
1230067                -
Name: 계약기간, dtype: object

In [12]:
apt['갱신요구권 사용'].value_counts()

-     1932473
사용      91718
Name: 갱신요구권 사용, dtype: int64

In [13]:
print(len(apt[apt['계약기간'] == "-"]))
print(len(apt[apt['계약기간'] != '-']))

1559139
465052


In [14]:
apt['계약기간'].sample(10)

1746216    -
741858     -
134714     -
25715      -
518902     -
163722     -
20315      -
192146     -
533200     -
195830     -
Name: 계약기간, dtype: object

In [15]:
apt['계약기간'].value_counts()

-                1559139
202202~202402      20849
202302~202502      19875
202303~202503      17026
202308~202508      15983
                  ...   
202201~202509          1
202105~202203          1
202107~202810          1
202105~202601          1
202210~202603          1
Name: 계약기간, Length: 1737, dtype: int64

In [16]:
apt.replace({'계약기간' : {'-' : '0~0'}},inplace=True) # 공란 값 변경

In [17]:
apt['계약기간'].value_counts()

0~0              1559139
202202~202402      20849
202302~202502      19875
202303~202503      17026
202308~202508      15983
                  ...   
202201~202509          1
202105~202203          1
202107~202810          1
202105~202601          1
202210~202603          1
Name: 계약기간, Length: 1737, dtype: int64

In [18]:
apt['계약기간'].sample(20)

1336540              0~0
2019806              0~0
1228402              0~0
695075               0~0
1893153    202308~202508
318384               0~0
521879               0~0
1365232              0~0
1855710    202309~202509
982870               0~0
1982349              0~0
953326               0~0
2015197    202304~202504
574515               0~0
851415               0~0
1867774    202302~202502
632581               0~0
79697                0~0
857390               0~0
1674876    202204~202404
Name: 계약기간, dtype: object

In [20]:
apt[['계약시작일','계약종료일']] = apt['계약기간'].str.split("~", expand = True)

In [21]:
apt[['계약시작일','계약종료일']].sample(5)

,계약시작일,계약종료일
447608,0,0
839365,0,0
1418988,202108,202308
1439004,202202,202402
679237,0,0


In [63]:
apt.sample(5)['계약기간'] #1916178

1916177    202211~202411
1157067                -
1020455                -
155534                 -
1120138                -
Name: 계약기간, dtype: object

In [22]:
apt[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]]

,sido,sigungu,ilbangu,eupmyeondong,ri_roadname
0,서울특별시,강남구,,개포동,
1,서울특별시,강남구,,개포동,
2,서울특별시,강남구,,개포동,
3,서울특별시,강남구,,개포동,
4,서울특별시,강남구,,개포동,
...,...,...,...,...,...
2024444,서울특별시,중랑구,,중화동,
2024445,서울특별시,중랑구,,중화동,
2024446,서울특별시,중랑구,,중화동,
2024447,서울특별시,중랑구,,중화동,


In [23]:
apt['eupmyeondong'].value_counts()
# apt.sample(5)

상계동      72568
신정동      50760
잠실동      44482
목동       37359
서초동      36957
         ...  
보문동7가        6
광희동1가        6
남산동2가        5
주성동          3
주교동          3
Name: eupmyeondong, Length: 342, dtype: int64

In [18]:
apt.columns # ri_roadname 리

Index(['시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금 (만원)', '종전계약 월세 (만원)', '법정동코드', '산', '토지고유번호', 'sido',
       'sigungu', 'ilbangu', 'eupmyeondong', 'ri_roadname', '계약시작일', '계약종료일'],
      dtype='object')

In [24]:
apt['갱신요구권 사용'].value_counts()

-     1932473
사용      91718
Name: 갱신요구권 사용, dtype: int64

In [72]:
apt['도로명'].astype(str).str.len().value_counts()

7     483832
9     455453
8     391486
10    313174
6     200060
11    107246
12     31896
1      19293
13     10694
5       7968
14      1273
4        778
15       540
3        467
2         30
16         1
Name: 도로명, dtype: int64

In [66]:
yeon[yeon['층'].astype(str).str.len() == 3]['층']

# yeon['층'].astype(int)

0          2.0
1          5.0
2          3.0
3          4.0
4          3.0
          ... 
1073934    3.0
1073935    4.0
1073936    3.0
1073937    2.0
1073938    2.0
Name: 층, Length: 1037487, dtype: float64

In [58]:
yeon['층'].astype(str).str.len().value_counts() # 읍면동 최대 6자리 시군구 최대 4자리 시도 5자리 

3    1037487
4      36443
5          9
Name: 층, dtype: int64

In [53]:
apt['계약구분'].value_counts()

-     1543035
신규     310836
갱신     170320
Name: 계약구분, dtype: int64

In [49]:
apt['전용면적(㎡)'].astype(str).str.len().value_counts() 

5    1364694
6     342260
4     196935
7     108121
8      12162
3         19
Name: 전용면적(㎡), dtype: int64

In [41]:
cursor.execute("""
               SELECT TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,COLUMN_COMMENT
               FROM INFORMATION_SCHEMA.COLUMNS
               WHERE TABLE_SCHEMA = 'capstone'
               AND TABLE_NAME   = 'rent_price';
               """)

In [42]:
result = pd.DataFrame(cursor.fetchall())

In [43]:
result

,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,COLUMN_COMMENT
0,capstone,rent_price,address,토지의 고유한 위치를 지정하는 토지고유번호 \n시 + 시군구(일반구) + 읍면동 +...
1,capstone,rent_price,sido,시
2,capstone,rent_price,sigungu,시군구
3,capstone,rent_price,ilbangu,일반구
4,capstone,rent_price,eupmyeondong,읍면동
5,capstone,rent_price,ri,리
6,capstone,rent_price,main_num,본번
7,capstone,rent_price,sub_num,부번
8,capstone,rent_price,is_mountain,산 여부
9,capstone,rent_price,rent_type,전월세구분


In [48]:
apt.columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금 (만원)', '종전계약 월세 (만원)', '법정동코드', '산', '토지고유번호', 'sido',
       'sigungu', 'ilbangu', 'eupmyeondong', 'ri_roadname', '계약시작일', '계약종료일'],
      dtype='object')

In [79]:
print(result.COLUMN_COMMENT.values)
# 토지고유번호,sido,sigungu,ilbangu,eupmyeondong,ri,본번,부번,산여부,전월세구분,전용면적,계약일자,보증금,월세,종전계약 보증금,종전계약 월세, 건물명,도로명,층,건축년도, 계약 시작일, 계약 종료일, 계약 구분
insert_df = apt[["토지고유번호","sido","sigungu","ilbangu","eupmyeondong","ri_roadname","본번","부번",'산',"전월세구분","전용면적(㎡)","계약년월","계약일","보증금(만원)","월세(만원)","종전계약 보증금 (만원)",'종전계약 월세 (만원)',"단지명","도로명","층","건축년도","계약시작일",'계약종료일','계약구분']]

['토지의 고유한 위치를 지정하는 토지고유번호 \n시 + 시군구(일반구) + 읍면동 + 리 + 산 여부 + 본번 + 부번 (19자리)'
 '시' '시군구' '일반구' '읍면동' '리' '본번' '부번' '산 여부' '전월세구분' '전용면적(㎡)' '계약일자'
 '보증금\n' '월세' '종전계약 보증금' '종전계약 월세' '건물 명' '도로명' '층' '건축년도' '계약 시작일\n'
 '계약 종료일' '계약 구분( 0 : 기존, 1 : 신규, 2 : 갱신)' '갱신요구권 ( 0 : 미 사용, 1 : 사용)']


In [81]:
insert_df['본번'] = [str(int(round(float(i),0))).zfill(4) for i in insert_df['본번']]
insert_df['부번'] = [str(int(round(float(i),0))).zfill(4) for i in insert_df['부번']]

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_68947/3750755126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insert_df['본번'] = [str(int(round(float(i),0))).zfill(4) for i in insert_df['본번']]
/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_68947/3750755126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insert_df['부번'] = [str(int(round(float(i),0))).zfill(4) for i in insert_df['부번']]


In [ ]:
def inserter(row, data_type, data_len):
    
    '''
    각 row 별로 insert 작업 수행

    row : DataFrame의 row
    jdbc : JDBC 객체
    data_type : 데이터 종류 (예 : '아파트-전세', '아파트-매매')
    data_len : 총 데이터 수
    '''
    insert_tuple = (
        None if pd.isna(row['토지고유번호']) else row['토지고유번호'],  # address_id
        row['시도'],
        row['시군구'],
        row['일반구'],
        row['읍면동'],
        row['리'],
        2 if '산' in str(row['번지']) else 1,  # 산여부(1:일반, 2:산)
        None if pd.isna(row['본번']) else str(row['본번']),  # num1
        None if pd.isna(row['부번']) else str(row['부번']),  # num2
        row['도로명'],  # road_name
        None if pd.isna(row['단지명']) else row['단지명'],  # apart_name
        '아파트',  # property_type(주택유형)
        None if pd.isna(row['층']) else str(int(row['층'])),  # floor_number(층 수)
        row['전용면적(㎡)'],  # exclusive_area(전용면적)
        str(row['계약날짜']),  # contract_date(계약날짜)
        str(int(row['거래금액(만원)'].replace(',', '')) * 10000),  # price(거래금액)
        None if pd.isna(row['거래유형']) else row['거래유형'],
        None if pd.isna(row['중개사소재지']) or row['중개사소재지'] == '-' else
        row['중개사소재지'],
        None if pd.isna(row['건축년도']) else str(int(row['건축년도'])),
        1,
        None if pd.isna(row['해제사유발생일']) else str(int(row['해제사유발생일'])), # 해제사유일
        row['cFlag'], # 해제사유 취소여부 플래그
        row['hash'] # 해시 값
        )
    index = row.name + 1
    try:
        if index % (data_len // 10) == 0:
            print(f'{data_type} : {index/data_len*100:.2f}% a_id INSERT 완료')
    except:
        pass
    

In [ ]:
import re
import json


class AddressParser:
    '''
    주소를 각 명칭으로 나누거나
    토지고유번호를 매칭해주는 class
    '''
    __jibun_search_query = '''
        SELECT {}
        FROM addresses 
        WHERE sido = %s 
        AND sigungu = %s 
        AND ilbangu = %s 
        AND eupmyeondong = %s 
        AND ri = %s 
        AND is_mountain = %s 
        AND num1 = %s
        AND {}
    '''
    __road_search_query = '''
        SELECT {}
        FROM addresses 
        WHERE sido = %s 
        AND sigungu = %s 
        AND ilbangu = %s 
        AND road_name = %s
        AND road_num1 = %s
        AND road_num2 = %s
    '''
    __search_by_aid = '''
        SELECT {}
        FROM addresses
        WHERE id = %s
    '''

    def __init__(self):
        '''
        주소를 불러와
        sido, sigungu, ilbangu, eupmyeondong, ri_roadname으로 나누는 작업을 해주는 class
        '''
        self.re1 = re.compile("^(수원|고양|용인|창원|성남|청주|전주|천안|안산|안양|포항)")
        self.re2 = re.compile("([읍면동]|[0-9]가|시장북로|세종로|남성로)$")

    def parse(self, address, how='jibun'):
        '''
        주소를 받고 how 값에 따라 지번 또는 도로명 주소로 값을 return하는 메소드

        address : 주소
        how : 지번으로 parsing 또는 도로명으로 parsing ('jibun', 'road')
        '''
        parts = address.split()
        sido = ""
        sigungu = ""
        ilbangu = ""
        eupmyeondong = ""
        ri_roadname = ""
        for idx, part in enumerate(parts):
            if idx == 0:
                sido = part
                continue
            
            if part.endswith("시") or part.endswith("군"):
                sigungu = part
                continue
            
            if part.endswith("구"):
                if idx == 1:
                    if len(part) >= 4 and re.search(self.re1, part):
                        sigungu = part[0:2] + "시"
                        ilbangu = part[2:]
                    else:
                        sigungu = part
                else:
                    ilbangu = part
                continue
            
            if re.search(self.re2,
                         part) and eupmyeondong == '' and how == 'jibun':
                eupmyeondong = part
                continue
                
            if part.endswith("리") and how == 'jibun':
                ri_roadname = part
                continue
            
            if (part.endswith('로') or part.endswith('길')
                ) and how == 'road' and ri_roadname == '':
                ri_roadname = part
                continue
            
            if re.search(r"([가-힣]{2,})(\([^0-9가-힣a-zA-Z]{1,}\))", part):
                ri_roadname = part
                continue

        return sido, sigungu, ilbangu, eupmyeondong, ri_roadname

    def get_address_id(self,
                       addr,
                       http,
                       jdbc,
                       silent=False,
                       additional_columns=None):
        '''
        지번 또는 도로명 주소로 구성 되어있는 주소를 parsing하여 토지 고유번호 매칭.
        깔끔하게 구성한 주소여야 함.
        대량의 주소를 변경하는 경우로 예상하여 http나 jdbc 객체를 받게끔 정의

        addr : 주소
        http : urllib3 PoolManager 객체
        jdbc : JDBC 객체
        silent : 못 찾았을 때, 출력값 보여주기 여부
        additional_columns : addresses에서 id 값 뿐만 아니라 더 검색할 columns 값들
        '''
        search_columns = 'id, '
        if additional_columns is not None:
            for column in additional_columns:
                search_columns += column + ', '
        search_columns = search_columns[:-2]

        def search_road(addr, splitted_addr):
            '''
            주소 형태가 도로명 주소인 경우라고 판단 할 때, 매칭하는 코드
            먼저 postcodify에서 검색 후 없으면 addresses에서 검색

            addr : 주소
            splitted_addr : 분리된 주소
            '''
            number = addr.split()[-1]
            road_name = splitted_addr[-1] + ' ' + number

            number = number.split('-')
            num1 = number[0]
            num2 = number[1] if len(number) == 2 else 0

            r = http.request('GET',
                             'http://192.168.20.2/postcodify/lib/search.php',
                             fields={'q': addr})

            if r.status == 200:
                results = json.loads(r.data.decode('utf8'))['results']
                filtered = list(
                    filter(lambda x: x['ko_doro'] == road_name, results))

                if len(filtered) == 1:
                    a_id = filtered[0]['address_id']
                    res = jdbc.fetch_data(
                        AddressParser.__search_by_aid.format(search_columns),
                        (a_id, ),
                        ret_type='raw')
                    if len(res) == 1:
                        if additional_columns is None:
                            return res[0][0]
                        return res[0]

                res = jdbc.fetch_data(
                    AddressParser.__road_search_query.format(search_columns),
                    splitted_addr[:3] + (splitted_addr[-1], num1, num2),
                    ret_type='raw')
                if len(res) == 1:
                    if additional_columns is None:
                        return res[0][0]
                    return res[0]

                return None
            raise Exception('도로명 주소 서버 통신 실패')

        def search_jibun(addr, splitted_addr):
            '''
            주소 형태가 지번 주소인 경우라고 판단 할 때, 매칭하는 코드

            addr : 주소
            splitted_addr : 분리된 주소
            '''
            number = addr.split()[-1]
            if number[0] == '산':
                number = number[1:]
                is_mountain = 2
            else:
                is_mountain = 1
            number = number.split('-')
            num1 = number[0]
            num2_case = f'num2 = {number[1]}' if len(
                number) == 2 and number[1].isdigit() else '(num2 IS NULL or num2 = 0)'

            res = jdbc.fetch_data(AddressParser.__jibun_search_query.format(
                search_columns, num2_case),
                                  splitted_addr + (is_mountain, num1),
                                  ret_type='raw')
            if len(res) == 1:
                if additional_columns is None:
                    return res[0][0]
                return res[0]
            return None

        match = re.compile(r'\s+산{0,1}\d{1,}-?\d{0,}').search(addr)

        if match is None:
            return None

        used_addr = addr[:match.end()]

        road_splited_addr = self.parse(used_addr, how='road')
        jibun_splited_addr = self.parse(used_addr, how='jibun')
        addr_type = 'road'
        if road_splited_addr[-1] == '':
            addr_type = 'jibun'

        if addr_type == 'road':
            result = search_road(used_addr, road_splited_addr)
            if result is None:
                result = search_jibun(used_addr, jibun_splited_addr)
        else:
            result = search_jibun(used_addr, jibun_splited_addr)
            if result is None:
                result = search_road(used_addr, road_splited_addr)
        if result is None and not silent:
            print(f'"{used_addr}" 주소 하나로 결정 불가')
        return result
